In [1]:
from google.colab import drive
drive.mount('/content/drive')
DRIVE_PATH = '/content/drive/MyDrive/dreambooth'
!mkdir -p '{DRIVE_PATH}'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
%cd '{DRIVE_PATH}'
!rm -rf diffusers
!git clone https://github.com/huggingface/diffusers
%cd '{DRIVE_PATH}'

/content/drive/MyDrive/dreambooth
Cloning into 'diffusers'...
remote: Enumerating objects: 85569, done.
remote: Counting objects: 100% (443/443), done.
remote: Compressing objects: 100% (270/270), done.
remote: Total 85569 (delta 345), reused 173 (delta 170), pack-reused 85126 (from 3)
Receiving objects: 100% (85569/85569), 62.16 MiB | 12.69 MiB/s, done.
Resolving deltas: 100% (62701/62701), done.
Updating files: 100% (1953/1953), done.
/content/drive/MyDrive/dreambooth/diffusers
Processing /content/drive/MyDrive/dreambooth/diffusers
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for diffusers: filename=diffusers-0.33.0.dev0-py3-none-any.whl size=3544295 sha256=394793380fd0624197d8260e6c103e5e015c60b4454b07abf72e41e8309afeca
  Stored in directory: /tmp/pip-ephem-wheel-cache-35i7tt9h/wheels/23/c6/82/6c9aa22451cbda3ce2e472f7cef848f4d4a5d08d21f10bf910
Successfully built diffusers
  Atte

In [4]:
%cd '{DRIVE_PATH}'
%cd diffusers
!pip install .
%cd examples/dreambooth
!pip install -r requirements.txt
%cd '{DRIVE_PATH}'

/content/drive/MyDrive/dreambooth
/content/drive/MyDrive/dreambooth/diffusers
Processing /content/drive/MyDrive/dreambooth/diffusers
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for diffusers: filename=diffusers-0.33.0.dev0-py3-none-any.whl size=3544295 sha256=135b0cd345341743ac8063535c57015a09bd7bef1cff0674fe9d0351842865f5
  Stored in directory: /tmp/pip-ephem-wheel-cache-u5tj754u/wheels/23/c6/82/6c9aa22451cbda3ce2e472f7cef848f4d4a5d08d21f10bf910
Successfully built diffusers
  Attempting uninstall: diffusers
    Found existing installation: diffusers 0.33.0.dev0
    Uninstalling diffusers-0.33.0.dev0:
      Successfully uninstalled diffusers-0.33.0.dev0
/content/drive/MyDrive/dreambooth/diffusers/examples/dreambooth
/content/drive/MyDrive/dreambooth


In [3]:
!pip install bitsandbytes

In [5]:
!accelerate config default

Configuration already exists at /root/.cache/huggingface/accelerate/default_config.yaml, will not override. Run `accelerate config` manually or pass a different `save_location`.


In [10]:
#@title Importing training images
import os

%cd '{DRIVE_PATH}'
!rm -rf training_images
!mkdir -p training_images
%cd training_images
!git init
!git pull https://github.com/google/dreambooth.git
%cd '{DRIVE_PATH}'

/content/drive/MyDrive/dreambooth
/content/drive/MyDrive/dreambooth/training_images
hint: Using 'master' as the name for the initial branch. This default branch name
hint: is subject to change. To configure the initial branch name to use in all
hint: of your new repositories, which will suppress this warning, call:
hint: 
hint: 	git config --global init.defaultBranch <name>
hint: 
hint: Names commonly chosen instead of 'master' are 'main', 'trunk' and
hint: 'development'. The just-created branch can be renamed via this command:
hint: 
hint: 	git branch -m <name>
Initialized empty Git repository in /content/drive/MyDrive/dreambooth/training_images/.git/
remote: Enumerating objects: 212, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 212 (delta 5), reused 1 (delta 1), pack-reused 206 (from 1)
Receiving objects: 100% (212/212), 106.84 MiB | 15.95 MiB/s, done.
Resolving deltas: 100% (10/10), done.
From https://github.com/googl

In [7]:
#@title Training
%cd '{DRIVE_PATH}'
import re
import os

!rm -rf output_model
!mkdir -p output_model
!rm -rf class_images
!mkdir -p class_images

itr=0
for root, dirs, files in os.walk("training_images/dataset"):
  for dir_name in dirs:
    source_folder_path = os.path.join(root, dir_name)
    class_name = re.sub(r'[0-9]','',dir_name).replace("_"," ")

    MODEL_NAME="CompVis/stable-diffusion-v1-4"
    INSTANCE_DIR=source_folder_path
    OUTPUT_DIR=f"./output_model/{dir_name}"
    CLASS_DIR=f"./class_images/{dir_name}"

    print(f"--------------------Training {dir_name}--------------------")

    !accelerate launch diffusers/examples/dreambooth/train_dreambooth.py \
      --pretrained_model_name_or_path="{MODEL_NAME}"  \
      --instance_data_dir="{INSTANCE_DIR}" \
      --output_dir="{OUTPUT_DIR}" \
      --class_data_dir="{CLASS_DIR}" \
      --with_prior_preservation --prior_loss_weight=1.0 \
      --instance_prompt="a photo of sks {class_name}" \
      --class_prompt="a photo of {class_name}" \
      --resolution=128 \
      --train_batch_size=1 \
      --gradient_accumulation_steps=1 \
      --use_8bit_adam \
      --learning_rate=5e-6 \
      --lr_scheduler="constant" \
      --lr_warmup_steps=0 \
      --num_class_images=10 \
      --max_train_steps=100
    itr += 1
    if (itr == 5):
       break

/content/drive/MyDrive/dreambooth
--------------------Training backpack--------------------
2025-04-04 14:58:39.255528: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1743778719.286809    5341 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1743778719.298324    5341 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-04 14:58:39.325345: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
04/04/2025 14:58:43 - INFO - __ma

In [8]:
!pip install git+https://github.com/openai/CLIP.git
!pip install pytorch_fid

  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-08ao1pto
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-08ao1pto
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369490 sha256=51af398033e0b821680ca6d8965396cef3ea85ef1bfb84aeabe88510c016476b
  Stored in directory: /tmp/pip-ephem-wheel-cache-wfe7qhgr/wheels/3f/7c/a4/9b490845988bf7a4db33674d52f709f088f64392063872eb9a
Successfully built clip


In [2]:
import clip
import numpy as np

def compute_clip_similarity(images, prompt, clip_model, clip_preprocess, device="cuda"):
    """Compute average CLIP cosine similarity between generated images and a prompt."""
    text_tokens = clip.tokenize([prompt]).to(device)
    with torch.no_grad():
        text_features = clip_model.encode_text(text_tokens)
    text_features = text_features / text_features.norm(dim=-1, keepdim=True)
    similarities = []
    for img in images:
        image_input = clip_preprocess(img).unsqueeze(0).to(device)
        with torch.no_grad():
            image_features = clip_model.encode_image(image_input)
        image_features = image_features / image_features.norm(dim=-1, keepdim=True)
        sim = (image_features @ text_features.T).item()
        similarities.append(sim)
    return np.mean(similarities)
def generate_images_for_class(prompt, pipe ,output_dir ,num_images = 5):
    """
    Generate images using Stable Diffusion for a given prompt and save them.
    """
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    generated_images = []
    for i in range(num_images):
        image = pipe(prompt).images[0].resize((256, 256))
        image_path = os.path.join(output_dir, f"generated_{i}.png")
        image.save(image_path)
        generated_images.append(image)
    return generated_images

In [3]:
from pytorch_fid import fid_score
from PIL import Image

def fid_score_calc(real_img_dir,gen_img_dir):
    # from fid_score import calculate_fid_given_paths
    # Calculate FID score
    fid_score_val = fid_score.calculate_fid_given_paths([real_img_dir, gen_img_dir], batch_size=2, device="cuda",dims=2048)
    return fid_score_val


def resize_images(image_dir, target_size=(256, 256)):
  valid_extensions = ('.png', '.jpg', '.jpeg')
  for img_name in os.listdir(image_dir):
      if img_name.endswith(valid_extensions):
          img_path = os.path.join(image_dir, img_name)
          img = Image.open(img_path).resize(target_size)
          img.save(img_path)

In [4]:
%cd '{DRIVE_PATH}'
from diffusers import StableDiffusionPipeline
import torch
import os
from pytorch_fid import fid_score
import re

!rm -rf /content/generated_images
!mkdir -p /content/generated_images

for root, dirs, files in os.walk("output_model"):
  for dir_name in dirs:
    model_id = f"./output_model/{dir_name}"
    pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16).to("cuda")
    class_name = re.sub(r'[0-9]','',dir_name).replace("_"," ")
    prompt = f"a photo of sks {class_name}"
    out_dir = f"/content/generated_images/{dir_name}"
    generated_images = generate_images_for_class(prompt,pipe,out_dir,num_images=1)
    resize_images(f"{DRIVE_PATH}/training_images/dataset/{dir_name}")
    clip_model, clip_preprocess = clip.load("ViT-L/14", device="cuda")
    clip_score = compute_clip_similarity(generated_images, prompt, clip_model, clip_preprocess)
    print(f"CLIP SCORE for {dir_name}: {clip_score}")
  break
%cd '{DRIVE_PATH}'

/content/drive/MyDrive/dreambooth


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

CLIP SCORE for backpack: 0.2371826171875


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

...

  0%|          | 0/50 [00:00<?, ?it/s]

CLIP SCORE for berry_bowl: 0.2431640625


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

CLIP SCORE for can: 0.25537109375
/content/drive/MyDrive/dreambooth
